# Tempora - Climate Analysis 

Himesh Dulal (20), Avipsa Parajuli (37), Aayusha Shrestha (50), Pratistha Thapa (57), Abhinav Lamsal (68)

## <b>Summary </b>
The project uses numerical methods like Interpolation, Linear Regression, Polynomial Fit + Extrapolation and Newton Raphson Method for analysis and predicting the climate data of data set provided as per csv.

## 1. Loading the CSV Data

In [7]:
import pandas as pd

def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    df = df.sort_values(['Year', 'Month']).reset_index(drop=True)
    return df


ModuleNotFoundError: No module named 'pandas'

In [ ]:
def linear_interpolate_series(series):
    result = series.copy()
    for i in range(len(series)):
        if pd.isna(series[i]):
            # Find previous known value
            prev_index = i - 1
            while prev_index >= 0 and pd.isna(series[prev_index]):
                prev_index -= 1

            # Find next known value
            next_index = i + 1
            while next_index < len(series) and pd.isna(series[next_index]):
                next_index += 1

            if prev_index >= 0 and next_index < len(series):
                # Perform linear interpolation
                x0, y0 = prev_index, series[prev_index]
                x1, y1 = next_index, series[next_index]
                x = i
                result[i] = y0 + (x - x0) * (y1 - y0) / (x1 - x0)

    return result

In [ ]:
def clean_data(df):
    print("Data with Missing Values:")
    print(df[['Year', 'Month', 'Temp_2m', 'Precip']][df['Temp_2m'].isna() | df['Precip'].isna()])

    cleaned_df = df.copy()

    cleaned_df['Temp_2m'] = linear_interpolate_series(cleaned_df['Temp_2m'])
    cleaned_df['Precip'] = linear_interpolate_series(cleaned_df['Precip'])

    print("\n After Cleaning:")
    print(cleaned_df[['Year', 'Month', 'Temp_2m', 'Precip']])

    # Plot before vs after
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(df['Temp_2m'], label='Original Temp')
    plt.plot(cleaned_df['Temp_2m'], label='Cleaned Temp', linestyle='--')
    plt.title('Temperature - Before vs After Interpolation')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(df['Precip'], label='Original Rain')
    plt.plot(cleaned_df['Precip'], label='Cleaned Rain', linestyle='--')
    plt.title('Rainfall - Before vs After Interpolation')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return cleaned_df

In [ ]:
cleaned_df = clean_data(df)

In [ ]:
def visualize_trends(df):
    # Group by year
    yearly_df = df.groupby('Year').agg({
        'Temp_2m': 'mean',
        'Precip': 'sum'
    }).reset_index()

    plt.figure(figsize=(10, 5))

    plt.plot(yearly_df['Year'], yearly_df['Temp_2m'], color='orange', marker='o', label='Avg Temp (°C)')
    plt.plot(yearly_df['Year'], yearly_df['Precip'], color='blue', marker='s', label='Total Rainfall (mm)')

    plt.title('Climate Trends in Kathmandu (2000–2019)')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_trends(cleaned_df)

In [4]:
district="Kathmandu"
filepath=f"./data/final_data/{district}.csv"

df = load_and_prepare_data(filepath)

,Year,Month,District,Temp_2m,Precip,MaxTemp_2m,MinTemp_2m,Humidity_2m,Pressure
0,2000,1,Kathmandu,9.266452,0.89,16.704516,4.582903,2.477097,84.094516
1,2000,2,Kathmandu,9.134828,1.36,16.620690,3.712069,2.516897,83.967586
2,2000,3,Kathmandu,14.473226,NaN,22.110968,8.293548,2.431613,83.900968
3,2000,4,Kathmandu,20.670333,5.75,28.268000,13.678333,3.884000,83.694667
4,2000,5,Kathmandu,22.506452,58.19,28.208387,17.260000,10.369677,83.639032
...,...,...,...,...,...,...,...,...,...
232,2019,8,Kathmandu,21.316774,192.12,24.331290,18.727419,17.106452,83.564516
233,2019,9,Kathmandu,19.507333,246.45,22.374000,17.231667,15.529333,83.947000
234,2019,10,Kathmandu,16.337419,22.58,20.261935,13.549032,11.908710,84.242903
235,2019,11,Kathmandu,13.881000,34.15,18.631333,10.839333,8.598667,84.302000
